## Advanced RAG Demo: Agentic RAG with Langchain Agents

This notebook explores a more advanced RAG setup using Langchain Agents. An agent can use a set of tools (like our PDF retriever) to answer complex questions. This mimics the ReAct (Reason + Act) paradigm where the LLM can reason about what information it needs, act to retrieve it, and then use that information to form an answer.

We will:
1. Set up the PDF vector store as before.
2. Create a `Tool` from our retriever.
3. Initialize a Langchain Agent (e.g., a ReAct agent or a conversational agent that can use tools) with the Groq LLM.
4. Pose complex questions that might require the agent to decide to use the retrieval tool.
5. Observe the agent's thought process (intermediate steps).

### 1. Setup: Install Libraries and Import Modules

In [ ]:
# We might need langchain_experimental for some agent setups, or specific agent toolkits
!pip install -q langchain langchain-groq langchain-community pypdf faiss-cpu pypdf sentence-transformers langchain_experimental

In [ ]:
import os
import getpass
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Agent-specific imports
from langchain.agents import Tool, AgentExecutor
from langchain.agents.react.agent import create_react_agent # A common ReAct agent
# Or, for a more modern approach with LCEL:
from langchain import hub
from langchain.agents import create_tool_calling_agent # if Groq supports tool calling well
                                                  # otherwise, ReAct is more robust for general LLMs

from langchain.prompts import PromptTemplate # For ReAct agent prompt

### 2. Configure Groq API Key

In [ ]:
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

In [ ]:
os.makedirs("pdfs", exist_ok=True)

# Step 3: Download the PDF using requests
import requests

url = "https://cs229.stanford.edu/main_notes.pdf"
pdf_path = "pdfs/main_notes.pdf"

response = requests.get(url)
with open(pdf_path, "wb") as f:
    f.write(response.content)

print(f"PDF downloaded to: {pdf_path}")

### 3. Prepare PDF Document & Vector Store (Abbreviated - assuming it's done)

In [ ]:
chunks = []
vector_store = None

if os.path.exists(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    chunks = text_splitter.split_documents(documents)
    print(f"Split into {len(chunks)} chunks.")

    if chunks:
        embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
        embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
        print("Creating FAISS vector store...")
        # For speed in demo, let's try to load if exists, else create
        if os.path.exists("faiss_index_cs229_agent"):
             vector_store = FAISS.load_local("faiss_index_cs229_agent", embeddings, allow_dangerous_deserialization=True)
             print("Loaded FAISS index from disk.")
        else:
            vector_store = FAISS.from_documents(chunks, embeddings)
            vector_store.save_local("faiss_index_cs229_agent")
            print("FAISS vector store created and saved.")
else:
    print("PDF not found, agent will not have PDF tool.")

### 4. Initialize LLM and Create Retriever Tool

In [ ]:
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1) # Using a more capable model for agent tasks

tools = []
if vector_store:
    # Create a RetrievalQA chain to be used as a tool
    pdf_qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector_store.as_retriever(search_kwargs={'k': 3}),
        return_source_documents=True
    )

    # Define the tool
    pdf_retrieval_tool = Tool(
        name="Stanford_CS229_Notes_Retriever",
        func=lambda q: pdf_qa_chain.invoke({"query": q})['result'], # Agent expects string output from tool
        description="Useful for answering questions about machine learning concepts, algorithms, and theory based on the Stanford CS229 course notes. Input should be a specific question."
    )
    tools.append(pdf_retrieval_tool)
    print("PDF Retrieval tool created.")
else:
    print("PDF Retrieval tool not created as vector store is unavailable.")

### 5. Initialize the ReAct Agent

We'll use the `create_react_agent` constructor. This requires a prompt that includes placeholders for `tools`, `tool_names`, `input`, `agent_scratchpad`, and `chat_history` (optional).

In [ ]:
agent_executor = None
if tools:
    # Get the ReAct prompt from Langchain Hub
    # This prompt is engineered to guide the LLM in the ReAct cycle (Thought, Action, Observation)
    prompt = hub.pull("hwchase17/react")
    # print(prompt.template) # You can inspect the prompt template

    # Construct the ReAct agent
    agent = create_react_agent(llm, tools, prompt)

    # Create an agent executor
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True, # Shows the agent's thought process
        handle_parsing_errors=True, # Important for robustness
        max_iterations=5 # Prevent runaway agents
    )
    print("ReAct agent executor created.")
else:
    print("Agent not created as no tools are available.")

### 6. Run the Agent with Complex Questions

In [ ]:
if agent_executor:
    query1 = "What are the key differences between Principal Component Analysis (PCA) and Linear Discriminant Analysis (LDA) as described in the CS229 notes?"
    print(f"\n--- Agent Query 1: {query1} ---")
    try:
        response1 = agent_executor.invoke({"input": query1})
        print(f"\nAgent Response 1: {response1['output']}")
    except Exception as e:
        print(f"Error during agent execution: {e}")

    query2 = "Could you explain the bias-variance tradeoff and then tell me how it relates to model selection, using information from the CS229 notes?"
    print(f"\n--- Agent Query 2: {query2} ---")
    try:
        response2 = agent_executor.invoke({"input": query2})
        print(f"\nAgent Response 2: {response2['output']}")
    except Exception as e:
        print(f"Error during agent execution: {e}")

    # A question the agent should realize it cannot answer from the PDF tool
    query3 = "What is the current weather in Palo Alto?"
    print(f"\n--- Agent Query 3: {query3} ---")
    try:
        response3 = agent_executor.invoke({"input": query3})
        print(f"\nAgent Response 3: {response3['output']}")
    except Exception as e:
        print(f"Error during agent execution: {e}")
else:
    print("Cannot run agent queries as agent_executor is not available.")

### 7. Conclusion

This notebook demonstrated an agentic RAG system. The agent, powered by a Groq LLM, could reason about when to use its PDF retrieval tool to answer questions. The `verbose=True` setting allowed us to see the 'Thought:', 'Action:', 'Observation:' cycle, which is characteristic of ReAct agents. This approach enables handling more complex, multi-step queries where the LLM needs to actively seek information before formulating a final answer. For production, you'd want more sophisticated error handling and potentially more tools for the agent.